## 第十三週：基本圖學（中文）
**Autor**: 張芳瑜 <br>
**Date created**: 2021/11/30 <br>
**Last modified**: 2021/12/5

本週課程為 **基本圖學** ，主要會介紹圖的定義、性質，以及如何用套件呈現網絡圖。<br>
網絡圖有許多可以應用的情境，許多牽扯到「關聯」、「關係」都可以使用圖的方式來表達，像是：社交網路平台的好友關係、動物演生的關係、文章提到人物的關係...等。<br>
而本 jupyter notebook 使用 pyvis 來建立三國演義中人物的人物關係圖，並進一步進行操作、分析。
<br>
<br>
這次課程分為四個部分：1.介紹使用到的套件 2.將資料進行前處理 3.建立Network


## 大綱
1. 套件說明
1. 資料前處理
    + 2.1 資料描述
    + 2.2 Jieba中文斷詞、匯入字典
    + 2.3 篩選出要分析的人物
    + 2.4 計算人物間Correlation
1. 建立Network
    + 3.1 畫出人物關聯性社群網路圖

## 1. 套件說明
本次實作過程中使用到的相關套件，在課堂中我們將會簡單的展示這些套件的使用方式，關於各個套件的進階使用請同學參照其官網的說明。
+ pandas:提供高效能、簡易使用的資料格式與資料處理的函數。
+ jieba:中文斷詞套件
+ re:正規化表達式套件
+ IPython:互動式開發環境
+ numpy: 數據結構套件
+ pyvis:關係網路圖套件
+ permutations:對集合或字符串進行排序或排列的所有可能的組合

In [2]:
# colab
# !pip install pyvis

In [3]:
import pandas as pd
import jieba.analyse
import jieba
import re
import IPython
import numpy as np
from pyvis.network import Network
from itertools import permutations

## 2. 資料前處理
根據文字處理基本流程，我們需要將我們的資料進行資料前處理。
+ 資料收集：通常我們使用網路爬蟲技術來取得資料（網路爬蟲非本課程之目的），因此在此範例中，我們古騰堡文本來進行分析。
+ 資料清理：將文本內容轉為正規語句，例如：統一標點符號、去除特定標籤等等。
+ 斷詞、斷句：使用工具區分文章中的句子、詞彙，以便於進行分析。
+ 去除停用字：將與分析無關的詞彙去除，例如：語助詞、連接詞等等，來避免影響後續分析結果。

**2.1 資料描述**
+ 來源：古騰堡官網
+ 文本資料：三國演義文本

In [4]:
# 古騰堡下載三國演義文本 https://www.gutenberg.org/files/23950/23950-0.txt
# 將下載的txt檔讀入
with open(f'./raw_data/country.txt', 'r', encoding='UTF-8-sig') as f:
    data = f.read().splitlines()

data[:10]

['第一回：宴桃園豪傑三結義，斬黃巾英雄首立功',
 '',
 '\u3000\u3000詞曰：',
 '',
 '\u3000\u3000滾滾長江東逝水，浪花淘盡英雄。是非成敗轉頭空：青山依舊在，幾度夕陽紅。白',
 '髮漁樵江渚上，慣看秋月春風。一壺濁酒喜相逢：古今多少事，都付笑談中。',
 '',
 '\u3000\u3000話說天下大勢，分久必合，合久必分：周末七國分爭，并入於秦。及秦滅之後，楚',
 '、漢分爭，又并入於漢。漢朝自高祖斬白蛇而起義，一統天下。後來光武中興，傳至獻',
 '帝，遂分為三國。推其致亂之由，殆始於桓、靈二帝。桓帝禁錮善類，崇信宦官。及桓']

去除重複的句子、空行清除
+ pandas.unique : 此函式可以刪掉重複資料，並保留資料順序
+ tolist : 將其轉換為list
+ len()>0 : 代表空行會被清除

In [5]:
content = pd.unique(data).tolist()
content = list(filter(lambda x: len(x)>0 ,content))

In [6]:
content_df = pd.DataFrame({
    'text':content
})
content_df

,text
0,第一回：宴桃園豪傑三結義，斬黃巾英雄首立功
1,詞曰：
2,滾滾長江東逝水，浪花淘盡英雄。是非成敗轉頭空：青山依舊在，幾度夕陽紅。白
3,髮漁樵江渚上，慣看秋月春風。一壺濁酒喜相逢：古今多少事，都付笑談中。
4,話說天下大勢，分久必合，合久必分：周末七國分爭，并入於秦。及秦滅之後，楚
...,...
17420,白帝託孤堪痛楚！孔明六出祁山前，願以隻手將天補。何期歷數到此終，長星半夜落山
17421,塢！姜維獨憑氣力高，九伐中原空劬勞。鍾會鄧艾分兵進，漢室江山盡屬曹。丕、叡、
17422,芳、髦纔及奐，司馬又將天下交。受禪臺前雲霧起，石頭城下無波濤。陳留歸命與安樂
17423,，王侯公爵從根苗。紛紛世事無窮盡，天數茫茫不可逃。鼎足三分已成夢，後人憑弔空


取得三國演義的文本章節
+ DataFrame.apply() : 將函式應用到每一列
+ DataFrame.cumsum() : 用於累加值

In [7]:
def get_chapter(text):
    if re.search('第.*回(：|$)', text) == None:
    #「第」之後接0~無限個任意字元，「回」之後是冒號或者結束
        return 0
    else:
        return 1

In [8]:
content_df['chapter'] = content_df['text'].apply(get_chapter).cumsum(axis=0)
content_df
# 總共有120回

,text,chapter
0,第一回：宴桃園豪傑三結義，斬黃巾英雄首立功,1
1,詞曰：,1
2,滾滾長江東逝水，浪花淘盡英雄。是非成敗轉頭空：青山依舊在，幾度夕陽紅。白,1
3,髮漁樵江渚上，慣看秋月春風。一壺濁酒喜相逢：古今多少事，都付笑談中。,1
4,話說天下大勢，分久必合，合久必分：周末七國分爭，并入於秦。及秦滅之後，楚,1
...,...,...
17420,白帝託孤堪痛楚！孔明六出祁山前，願以隻手將天補。何期歷數到此終，長星半夜落山,120
17421,塢！姜維獨憑氣力高，九伐中原空劬勞。鍾會鄧艾分兵進，漢室江山盡屬曹。丕、叡、,120
17422,芳、髦纔及奐，司馬又將天下交。受禪臺前雲霧起，石頭城下無波濤。陳留歸命與安樂,120
17423,，王侯公爵從根苗。紛紛世事無窮盡，天數茫茫不可逃。鼎足三分已成夢，後人憑弔空,120


**2.2 Jieba中文斷詞、匯入字典**
斷詞引擎設定：使用 Jieba 當作斷詞系統。<br>

1. Jieba 原先內建為簡體字字典，因此我們匯入繁體字典，斷詞結果會比較好。<br>
2. 另外，在一些特殊主題裡，會有獨特的專有名詞，因此我們需要加入這些字詞進去。<br>
例如：三國演義為古典文學，其中包含專有人名與事物的說法與現在較不同，剛好搜狗引擎有提供專有字典，我們可以使用他們提供的專有字典來讓結巴斷詞更加精確。<br>

搜狗引擎也有提供不同領域的字典，同學如果在做其他文集時可以參考該網站提供的字典，幫助斷詞。<br>
以下為轉換搜狗引擎字典為結巴的使用者自訂字典格式步驟：
1. [搜狗引擎](https://pinyin.sogou.com/dict/) : 在此處搜尋專有字典（記得要以簡體字搜尋！）
1. [搜狗詞庫scel轉txt](https://www.toolnb.com/tools-lang-zh-TW/scelto.html) : 將檔案轉換為txt檔
1. [純文字檔案簡體轉繁體](https://txtconv.arpuli.com/) : 下載字體為簡體，將其轉換成繁體
* 補充macbook的txt檔處理方式：[Mac 如何創建 txt 檔案？](https://applealmond.com/posts/49630)

In [9]:
# 初始化斷詞引擎
jieba.set_dictionary('./dict/dict.txt')

# 這裡匯入三國演義的文物名稱，使其可以得到全部人物名稱
jieba.load_userdict('./dict/name_country.text')

Building prefix dict from /Users/changfangyu/Desktop/NSYSU/碩班/110 社群媒體/code/SMA_2021F/week12_igraph/dict/dict.txt ...
Loading model from cache /var/folders/yw/v6fz89fx5pd827bs1070ywr00000gn/T/jieba.u923de77338df76f1c01398ae8e8973ec.cache
Loading model cost 0.332 seconds.
Prefix dict has been built successfully.


將斷詞結果存進 DataFrame
+ assign : 新增一個欄位「word」，值為content_df['text']進行jieba.lcut的結果
+ explode : jieba_lcut 的結果為list，因此用explode將list裡的每個值變單獨一行
+ drop : 將欄位「text」刪除，axis=0為對縱軸操作(column)，axis=1為對橫軸操作(row)

In [10]:
word_df = content_df.assign(word = content_df['text'].apply(jieba.lcut)).explode('word').drop(['text'], axis=1)
word_df

,chapter,word
0,1,第一
0,1,回
0,1,：
0,1,宴
0,1,桃園
...,...,...
17423,120,人
17423,120,憑弔
17423,120,空
17424,120,牢騷


匯入人物名稱並篩選出想分析的人物

In [11]:
# 匯入人物檔案
names = []

with open(f'./dict/name_country.text','r') as f:
    names = f.read().splitlines() #移除換行符號\n

In [12]:
# 篩選有在人物名稱裡的
word_df = word_df[(word_df['word'].isin(names) == True)]
word_df

,chapter,word
7,1,竇武
7,1,陳蕃
7,1,曹節
7,1,竇武
14,1,蔡邕
...,...,...
17416,120,張繡
17417,120,曹操
17421,120,姜維
17421,120,鍾會


計算每個字在不同章節出現的次數

In [13]:
word_df = word_df.assign(count = 1)
word_token = word_df.groupby(['chapter','word'])[['count']].sum().reset_index().sort_values('count', ascending=False)
word_token = word_token[ word_token['count'] > 1] #篩掉=1的
word_token

,chapter,word,count
555,15,孫策,61
2407,70,張郃,60
2249,65,馬超,57
2534,74,龐德,54
2980,88,孟獲,51
...,...,...,...
1547,43,曹氏,2
483,13,袁紹,2
1066,29,華佗,2
1065,29,程普,2


In [14]:
word_pairs = word_token.copy()
perms = word_pairs.groupby('chapter')['word'].apply(lambda s: pd.DataFrame(permutations(s, 2), columns=['term1', 'term2'])).reset_index()
perms = perms.drop('level_1', axis=1)
perms

,chapter,term1,term2
0,1,張角,盧植
1,1,張角,張飛
2,1,張角,劉焉
3,1,張角,鄒靖
4,1,張角,張寶
...,...,...,...
57425,120,張布,王渾
57426,120,張布,王戎
57427,120,張布,胡奮
57428,120,張布,陸凱


In [15]:
word_pair = perms.copy()
word_pair['n'] = 1
word_pair = word_pair.groupby(['term1','term2'])[['n']].sum().reset_index()
word_pair = word_pair.sort_values('n', ascending=False).reset_index().drop('index', axis=1)
word_pair.head(10)

,term1,term2,n
0,曹操,軍士,47
1,軍士,曹操,47
2,曹操,劉備,43
3,劉備,曹操,43
4,張飛,曹操,33
5,曹操,張飛,33
6,劉備,軍士,28
7,軍士,劉備,28
8,軍士,張飛,27
9,孫權,曹操,27


移除重複的人物組合（term1 & term2）

In [16]:
word_pair = word_pair[word_pair.index % 2 == 1]
word_pair

,term1,term2,n
1,軍士,曹操,47
3,劉備,曹操,43
5,曹操,張飛,33
7,軍士,劉備,28
9,孫權,曹操,27
...,...,...,...
37831,文欽,李鵬,1
37833,文欽,曹爽,1
37835,文欽,張翼,1
37837,文欽,尹大目,1


**2.4 計算人物間Correlation**
+ 斜對角(自己&自己)：correlation = 1
+ 上半部跟下半部會一樣

# 新增區段

In [17]:
chapter_word = pd.crosstab(word_pairs.chapter, word_pairs.word)
word_cor = chapter_word.copy()
word_cor = word_cor.reset_index()
correlation = word_cor.corr(method='pearson')
correlation

word,chapter,丁儀,丁原,丁夫人,丁奉,丁封,丁廙,丁斐,丁謐,丘建,...,黨均,龐德,龐德公,龐柔,龐統,龐羲,龐舒,龔景,龔起,龔都
word,,,,,,,,,,,,,,,,,,,,,
chapter,1.000000,0.048958,-0.152166,0.019848,0.260534,0.149520,0.048958,-0.006616,0.172885,0.154813,...,0.138935,0.051115,-0.067482,0.035726,-0.049186,-0.003970,-0.130995,-0.157459,0.096593,-0.148696
丁儀,0.048958,1.000000,-0.008403,-0.008403,-0.029121,-0.008403,1.000000,-0.008403,-0.011934,-0.008403,...,-0.008403,-0.024500,-0.008403,-0.008403,-0.024500,-0.008403,-0.008403,-0.008403,-0.008403,-0.014679
丁原,-0.152166,-0.008403,1.000000,-0.008403,-0.029121,-0.008403,-0.008403,-0.008403,-0.011934,-0.008403,...,-0.008403,-0.024500,-0.008403,-0.008403,-0.024500,-0.008403,-0.008403,-0.008403,-0.008403,-0.014679
丁夫人,0.019848,-0.008403,-0.008403,1.000000,-0.029121,-0.008403,-0.008403,-0.008403,-0.011934,-0.008403,...,-0.008403,0.342997,-0.008403,-0.008403,-0.024500,-0.008403,-0.008403,-0.008403,-0.008403,-0.014679
丁奉,0.260534,-0.029121,-0.029121,-0.029121,1.000000,0.288565,-0.029121,-0.029121,-0.041358,0.288565,...,0.288565,-0.084902,-0.029121,-0.029121,-0.084902,-0.029121,-0.029121,-0.029121,-0.029121,-0.050869
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
龐羲,-0.003970,-0.008403,-0.008403,-0.008403,-0.029121,-0.008403,-0.008403,-0.008403,-0.011934,-0.008403,...,-0.008403,0.342997,-0.008403,-0.008403,-0.024500,1.000000,-0.008403,-0.008403,-0.008403,-0.014679
龐舒,-0.130995,-0.008403,-0.008403,-0.008403,-0.029121,-0.008403,-0.008403,-0.008403,-0.011934,-0.008403,...,-0.008403,-0.024500,-0.008403,-0.008403,-0.024500,-0.008403,1.000000,-0.008403,-0.008403,-0.014679
龔景,-0.157459,-0.008403,-0.008403,-0.008403,-0.029121,-0.008403,-0.008403,-0.008403,-0.011934,-0.008403,...,-0.008403,-0.024500,-0.008403,-0.008403,-0.024500,-0.008403,-0.008403,1.000000,-0.008403,-0.014679


In [18]:
correlation.index.name = 'term1'
correlation = correlation.reset_index()
tidy = pd.DataFrame()
tidy = pd.melt(correlation, id_vars='term1', var_name='term2', value_name='correlation')
tidy

,term1,term2,correlation
0,chapter,chapter,1.000000
1,丁儀,chapter,0.048958
2,丁原,chapter,-0.152166
3,丁夫人,chapter,0.019848
4,丁奉,chapter,0.260534
...,...,...,...
648020,龐羲,龔都,-0.014679
648021,龐舒,龔都,-0.014679
648022,龔景,龔都,-0.014679
648023,龔起,龔都,-0.014679


移除自己跟自己的關聯

In [19]:
tidy = tidy.sort_values('correlation', ascending=False)
tidy = tidy[ tidy['term1'] != 'chapter']
tidy = tidy[ tidy['term2'] != 'chapter']
tidy = tidy[ tidy['term1'] != tidy['term2']] 
tidy

,term1,term2,correlation
210134,侯選,張衡,1.000000
212698,孫歆,張象,1.000000
212647,周旨,張象,1.000000
212536,伍延,張象,1.000000
211742,何進,張讓,1.000000
...,...,...,...
76788,曹操,司馬懿,-0.397706
128047,劉備,姜維,-0.452151
42019,姜維,劉備,-0.452151
252124,姜維,曹操,-0.553617


## 3. 建立Network

**3.1 畫出人物關聯性社群網路圖**<br>
+ 使用pyvis將前面所計算出來的correlation畫出網絡圖

correlation大於0.5

In [20]:
point_five = tidy[tidy['correlation']>0.5]
point_five.set_index = ('term1')
point_five

,term1,term2,correlation
210134,侯選,張衡,1.000000
212698,孫歆,張象,1.000000
212647,周旨,張象,1.000000
212536,伍延,張象,1.000000
211742,何進,張讓,1.000000
...,...,...,...
590567,紀靈,韓暹,0.500835
22663,呂虔,侯成,0.500835
99043,侯成,呂虔,0.500835
128149,夏侯霸,姜維,0.500112


In [ ]:
five_net = Network(height='750px', width='100%', bgcolor='#ffffff', font_color='black') #設定

sources = point_five['term1']
targets = point_five['term2']
weights = point_five['correlation']

edge_data = zip(sources, targets, weights)
for e in edge_data:
    src = e[0]
    dst = e[1]
    w = e[2]*10

    five_net.add_node(src, src, title=src)
    five_net.add_node(dst, dst, title=dst)
    five_net.add_edge(src, dst, value=w)

neighbor_map = five_net.get_adj_list()

for node in five_net.nodes:
    node['title'] += ' Neighbors:<br>' + '<br>'.join(neighbor_map[node['id']])
    node['value'] = len(neighbor_map[node['id']])

five_net.show('five_net.html')
# IPython.display.HTML(filename="./five_net.html")

縮小範圍以利於觀察：correlation大於0.9

In [50]:
point_nine = tidy[tidy['correlation']>0.9]
point_nine.set_index = ('term1')
point_nine

,term1,term2,correlation
210134,侯選,張衡,1.000000
212698,孫歆,張象,1.000000
212647,周旨,張象,1.000000
212536,伍延,張象,1.000000
211742,何進,張讓,1.000000
...,...,...,...
811,丁廙,丁儀,1.000000
1636,何苗,丁原,1.000000
2628,崔琰,丁夫人,1.000000
205967,關興,張苞,0.911388


In [51]:
nine_net = Network(height='750px', width='100%', bgcolor='#ffffff', font_color='black')

sources = point_nine['term1']
targets = point_nine['term2']
weights = point_nine['correlation']

edge_data = zip(sources, targets, weights)
for e in edge_data:
    src = e[0]
    dst = e[1]
    w = e[2]*10

    nine_net.add_node(src, src, title=src)
    nine_net.add_node(dst, dst, title=dst)
    nine_net.add_edge(src, dst, value=w)

neighbor_map = nine_net.get_adj_list()

for node in nine_net.nodes:
    node['title'] += ' Neighbors:<br>' + '<br>'.join(neighbor_map[node['id']])
    node['value'] = len(neighbor_map[node['id']])

nine_net.show('nine_net.html')
# IPython.display.HTML(filename="./nine_net.html")

我們可以根據上面的圖，來看每個人物間的關聯性，來分析他們之間是否存在什麼關係，例如：
+ 大喬 & 小喬：三國時期一對貌美如花的姐妹花，大喬嫁給孫策、小喬嫁給周瑜
+ 張角 & 張寶：張角自稱「天公將軍」，而弟弟張寶則自稱「地宮將軍」，拉起黃巾之亂的序幕。
